# Analyze EvoGFuzz

In [ ]:
from evogfuzz_formalizations.calculator import GRAMMAR, INITIAL_INPUTS, prop

In [ ]:
from evogfuzz import EvoGFuzz

In [ ]:
from importlib import reload  # Not needed in Python 2
import logging
reload(logging)
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s:  %(message)s"
)

evogfuzz = EvoGFuzz(
    grammar=GRAMMAR,
    inputs=INITIAL_INPUTS,
    prop=prop
)

In [ ]:
result = evogfuzz.execute()

In [ ]:
result = evogfuzz._get_latest_grammar()

In [ ]:
result

In [ ]:
from fuzzingbook.ProbabilisticGrammarFuzzer import ProbabilisticGrammarFuzzer

prob_fuzzer = ProbabilisticGrammarFuzzer(result)

for _ in range(100):
    print(prob_fuzzer.fuzz())

## Visualizing the Optimization

In [ ]:
from matplotlib.pyplot import plot

In [ ]:
plot([1,2], [2,1])

In [ ]:
result.keys()

In [ ]:
from copy import deepcopy
g = deepcopy(result)

features = set()

for rule in g:
    for child in g[rule]:
        features.add(f"{rule}->{child[1]['prob']}")

In [ ]:
display(features)

In [ ]:
gr = [g, g]

def _initialize(g):
    g = deepcopy(g)
    probabilities = dict()
    for rule in g:
        for child in g[rule]:
            feature = f"{rule}->{child[0]}"
            probabilities[feature] = []
    return probabilities



def parse_prob_grammars(grammar_list):
    probabilities = _initialize(grammar_list[0])

    for grammar in grammar_list:
        for rule in grammar:
            for child in grammar[rule]:
                feature = f"{rule}->{child[0]}"
                prob_list = probabilities[feature]
                if child[1]['prob'] is None:
                    prob_list.append(1.0)
                else:
                    prob_list.append(child[1]['prob'])
                probabilities[feature] = prob_list

    return probabilities


r = parse_prob_grammars(gr)

In [ ]:
display(r)

In [ ]:
all_prob_grammars_org = evogfuzz._probabilistic_grammars
all_prob_grammars = [g[0] for g in all_prob_grammars_org]

In [ ]:
# all_prob_grammars = [g[0] for g in all_prob_grammars]

In [ ]:
r = parse_prob_grammars(all_prob_grammars)

In [ ]:
display(r)

In [ ]:
from evogfuzz.evogfuzz_class import GrammarType
reduced = list(filter(lambda x: x[1] == GrammarType.LEARNED or GrammarType.MUTATED, all_prob_grammars_org))
reduced = [g[0] for g in reduced]
print(len(reduced))
dicts = parse_prob_grammars(reduced)

In [ ]:
display(dicts)

In [ ]:
import matplotlib.pyplot as plt

xvals = [i for i in range(len(reduced))]

f, ax = plt.subplots(1)

feature_disp = ['<function>->cos', '<function>->sqrt', '<maybe_minus>->-']

for feature in feature_disp:
    ax.plot(xvals, dicts[feature])


In [ ]:
display(evogfuzz._probabilistic_grammars)

### Plot Fitness Values

In [ ]:
fitness_values = [(x[2],x[1]) for x in all_prob_grammars_org[:]]
print("Original")
display(fitness_values)


from typing import Tuple, List

def clean_fitness_values(fitness_values: List[Tuple[int, GrammarType]]):
    for index, it in enumerate(fitness_values):
        if it[0] == -1:
            fitness_values[index] = (fitness_values[index-1][0], it[1])

    return fitness_values

res = clean_fitness_values(fitness_values)
print("Clean")
display(res)

In [ ]:
import matplotlib.pyplot as plt

xvals = [i for i in range(len(reduced))]

f, ax = plt.subplots(1)

feature_disp = ['<function>->cos', '<function>->sqrt', '<maybe_minus>->-']
# feature_disp = dicts.keys()

for feature in feature_disp:
    ax.plot(xvals, dicts[feature])

xvals = [i for i in range(len(res))]
yvals = [i[0] for i in res]

import numpy as np

def normalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

p = normalizeData(yvals)

ax.plot(xvals, p)

In [ ]:
data = zip({1,2,3}, [False,False,False])

In [ ]:
from fuzzingbook.ProbabilisticGrammarFuzzer import ProbabilisticGrammarMiner
from fuzzingbook.Parser import EarleyParser
from copy import deepcopy

new_g = deepcopy(GRAMMAR)
new_g['<maybe_minus>'] = ['', '-']

probabilistic_grammar_miner = ProbabilisticGrammarMiner(EarleyParser(new_g), log=True)
probabilistic_grammar = probabilistic_grammar_miner.mine_probabilistic_grammar(['sqrt(12)', 'tan(-1)'])

In [ ]:
display(probabilistic_grammar)

In [ ]:
p = EarleyParser(GRAMMAR)
for t in p.parse('sqrt(10)'):
    print(t)

In [ ]:
x = []
type(len(x))